In [27]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import numpy as np
import nltk
import pandas as pd
from ast import literal_eval
from collections import Counter
from scipy.optimize import fmin_l_bfgs_b
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import multiprocessing
import random
from tqdm import tqdm
import optimizeTopicVectors as ot
import time

In [29]:
# hoseong's class
from STMD import *
from ASUM import *
from ASUM_Embedding import *
from preprocess import *

In [30]:
with open('D:/Dropbox/2016-2/졸업논문/STMD/data/SentiWords-0.txt','r') as f:
    pos = f.readlines()
pos_seed = [word.strip() for word in pos]
with open('D:/Dropbox/2016-2/졸업논문/STMD/data/SentiWords-1.txt','r') as f:
    neg = f.readlines()
neg_seed = [word.strip() for word in neg]

In [31]:
# load data
# work_path = "/media/hs-ubuntu/data/dataset/MasterThesis/STMD_data/"
work_path = "E:/dataset/MasterThesis/STMD_data/"

data = pd.read_csv(work_path + "preprocess_complete_Electronics.csv")

In [35]:
def extract_sample(data, brand_name, count, ratio = 1, random_state = 42):
    brand = data[data['brand']==brand_name]
    pos_reviews = brand[brand.overall >= 4]
    neg_reviews = brand[brand.overall <= 2]
    if ratio == 1:
        pos_sample = pos_reviews.sample(count, random_state=random_state)
        neg_sample = neg_reviews.sample(count, random_state=random_state)
        df = pd.concat([pos_sample, neg_sample], axis=0)
    else:
        df = brand.sample(count * 2, random_state = random_state)
    df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))
    return df

In [37]:
# brand = ['Apple', 'Samsung','Canon']
# brand_df = data[data.brand.isin(brand)]
# brand_df.reset_index(drop=True, inplace=True)
# #긍정, 부정 반반씩
# pos_reviews = brand_df[brand_df.overall >= 4]
# neg_reviews = brand_df[brand_df.overall <= 2]
# pos_sample = pos_reviews.sample(3500, random_state=23)
# neg_sample = neg_reviews.sample(3500, random_state=42)
# df = pd.concat([pos_sample, neg_sample], axis=0)
# df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))
# df.reset_index(drop=True, inplace=True)

# apple, samsung, canon 1:1:1
apple = extract_sample(data, 'Apple', 500, ratio = 1)
samsung = extract_sample(data, 'Samsung', 500, ratio = 1)
canon = extract_sample(data, 'Canon', 500, ratio = 1)
df = pd.concat([apple, samsung, canon], axis=0)
df.reset_index(drop=True, inplace=True)

In [38]:
print(df[df.overall >= 4].shape)
print(df[df.overall <= 2].shape)

(1500, 12)
(1500, 12)


In [39]:
# prepare
sentence_list, sentiment_label, sentence_senti_label, \
pos_neg_sentence_indices, pos_neg_sentiment_label, numSentence = prepare(df)

documents, sentence_list_again, bigram, documents_label\
= bigram_and_sentence(sentence_senti_label, sentence_list, numSentence, max_vocab=5000, threshold = 5, min_count = 20)

In [40]:
len(documents)

27124

#### 그냥 ASUM

In [42]:
%%time
asum_path = "E:/dataset/MasterThesis/Models/ASUM_test1_"
asum = ASUM(pos_seed, neg_seed, numTopics=10, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
asum._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
asum.run(sentence_list_again, save_path=asum_path, print_iter=2, save_iter = 50, maxIters= 100)

Starting iteration 2 of 100
0.519
Starting iteration 4 of 100
0.518
Starting iteration 6 of 100
0.526
Starting iteration 8 of 100
0.523333333333
Starting iteration 10 of 100
0.518333333333
Starting iteration 12 of 100
0.528333333333
Starting iteration 14 of 100
0.531333333333
Starting iteration 16 of 100
0.549666666667
Starting iteration 18 of 100
0.546666666667
Starting iteration 20 of 100
0.552
Starting iteration 22 of 100
0.554333333333
Starting iteration 24 of 100
0.554
Starting iteration 26 of 100
0.554333333333
Starting iteration 28 of 100
0.551333333333
Starting iteration 30 of 100
0.559333333333
Starting iteration 32 of 100
0.558333333333
Starting iteration 34 of 100
0.553666666667
Starting iteration 36 of 100
0.551666666667
Starting iteration 38 of 100
0.545333333333
Starting iteration 40 of 100
0.552
Starting iteration 42 of 100
0.547333333333
Starting iteration 44 of 100
0.551333333333
Starting iteration 46 of 100
0.549
Starting iteration 48 of 100
0.549666666667
Starting it

In [43]:
%%time
asum_path = "E:/dataset/MasterThesis/Models/ASUM_test2_"
asum = ASUM(pos_seed, neg_seed, numTopics=10, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
asum._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
asum.run(sentence_list_again, save_path=asum_path, print_iter=2, save_iter = 50, maxIters= 100)

Starting iteration 2 of 100
0.499
Starting iteration 4 of 100
0.481
Starting iteration 6 of 100
0.479
Starting iteration 8 of 100
0.494
Starting iteration 10 of 100
0.493
Starting iteration 12 of 100
0.505666666667
Starting iteration 14 of 100
0.483333333333
Starting iteration 16 of 100
0.480666666667
Starting iteration 18 of 100
0.478666666667
Starting iteration 20 of 100
0.462666666667
Starting iteration 22 of 100
0.464
Starting iteration 24 of 100
0.460666666667
Starting iteration 26 of 100
0.451666666667
Starting iteration 28 of 100
0.456666666667
Starting iteration 30 of 100
0.454
Starting iteration 32 of 100
0.452
Starting iteration 34 of 100
0.445
Starting iteration 36 of 100
0.453
Starting iteration 38 of 100
0.447333333333
Starting iteration 40 of 100
0.442
Starting iteration 42 of 100
0.455333333333
Starting iteration 44 of 100
0.451333333333
Starting iteration 46 of 100
0.453333333333
Starting iteration 48 of 100
0.453333333333
Starting iteration 50 of 100
0.448
Starting sav

In [44]:
%%time
asum_path = "E:/dataset/MasterThesis/Models/ASUM_test3_"
asum = ASUM(pos_seed, neg_seed, numTopics=10, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
asum._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
asum.run(sentence_list_again, save_path=asum_path, print_iter=2, save_iter = 50, maxIters= 100)

Starting iteration 2 of 100
0.520666666667
Starting iteration 4 of 100
0.522
Starting iteration 6 of 100
0.523333333333
Starting iteration 8 of 100
0.501
Starting iteration 10 of 100
0.496
Starting iteration 12 of 100
0.506333333333
Starting iteration 14 of 100
0.506333333333
Starting iteration 16 of 100
0.514666666667
Starting iteration 18 of 100
0.518
Starting iteration 20 of 100
0.519
Starting iteration 22 of 100
0.522666666667
Starting iteration 24 of 100
0.526333333333
Starting iteration 26 of 100
0.530333333333
Starting iteration 28 of 100
0.526
Starting iteration 30 of 100
0.527666666667
Starting iteration 32 of 100
0.526
Starting iteration 34 of 100
0.520666666667
Starting iteration 36 of 100
0.525666666667
Starting iteration 38 of 100
0.519333333333
Starting iteration 40 of 100
0.514
Starting iteration 42 of 100
0.515333333333
Starting iteration 44 of 100
0.518333333333
Starting iteration 46 of 100
0.518
Starting iteration 48 of 100
0.508
Starting iteration 50 of 100
0.5123333

In [16]:
len(pos_neg_sentence_indices)

6000

In [17]:
len(pos_neg_sentiment_label)

5553

In [19]:
pos_neg_sentiment_label[:10]

[0, 0, 1, 0, 0, 1, 0, 1, 0, 0]